In [1]:
import requests

In [53]:
def fetch_data(url):
    headers = {
        # 'authority': f'',
        # 'content-type': 'application/json',
        # 'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="98", "Google Chrome";v="98"',
        # 'cache-control': 'no-cache',
        # 'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36',
        "Authorization": "Bearer"
    }
    # return requests.get(url,auth=('user','passwd'))
    requests.get(url,data=headers)

In [54]:
print(fetch_data("https://httpbin.org/#/Auth/get_bearer"))

None


None
